In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([9])


In [ ]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [ ]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [ ]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [ ]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.2748,  0.2376,  0.3712, -0.1699, -0.2439,  0.0532, -0.0371,  0.1538,
         -0.0094, -0.1740,  0.2095, -0.6243,  0.3648, -0.1237, -0.1177,  0.0855,
          0.2845,  0.1682, -0.0641, -0.0151],
        [-0.3629,  0.0033,  0.0723,  0.1154, -0.3555,  0.1198, -0.1025,  0.1631,
         -0.0213, -0.3054,  0.1941, -0.3311,  0.3408, -0.0967,  0.3764,  0.0124,
          0.4938,  0.0833,  0.1267, -0.2380],
        [-0.4689,  0.0575,  0.1115,  0.0102, -0.2984, -0.0775, -0.1733,  0.0583,
          0.0585, -0.1207, -0.0611, -0.4167,  0.3615,  0.0069,  0.2323,  0.0910,
          0.2642,  0.1500,  0.1978, -0.0499]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.2376, 0.3712, 0.0000, 0.0000, 0.0532, 0.0000, 0.1538, 0.0000,
         0.0000, 0.2095, 0.0000, 0.3648, 0.0000, 0.0000, 0.0855, 0.2845, 0.1682,
         0.0000, 0.0000],
        [0.0000, 0.0033, 0.0723, 0.1154, 0.0000, 0.1198, 0.0000, 0.1631, 0.0000,
         0.0000, 0.1941, 0.0000, 0.3408, 0.0000, 0.37

In [ ]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [ ]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [ ]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0060, -0.0257, -0.0068,  ..., -0.0055, -0.0281, -0.0272],
        [-0.0314, -0.0288,  0.0140,  ..., -0.0265, -0.0061, -0.0042]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0163,  0.0203], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0380,  0.0043, -0.0235,  ...,  0.0316, -0.0053,  0.0116],
        [ 0.0055, -0.0337, -0.0286,  ...,  0.0245, -0.0282, -0.0345]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 